In [1]:
import requests
import json
import os
import foursquare
import pandas as pd
from tqdm import tqdm


# Setup Foursquare API credentials 
api_key = os.environ.get('FOURSQUARE_API_KEY')

# Define the endpoint URL for the Foursqare request
endpoint = "https://api.foursquare.com/v3/places/search"

# Define the query parameters to include only the latitude and longitude fields
def foursquare_query(latitude, longitude, names):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000'
        }
    #Headers specific to FourSquare - used to specify wanted metadata - not typically found under all APIs. Found under https://location.foursquare.com/developer/reference/authentication
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
        }

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    return json_content

In [2]:
# Extract the relevant information from the Foursquare query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, location_name):
    json_content = foursquare_query(latitude, longitude)
    poi_list = []
    for poi in json_content['response']['groups'][0]['items']:
        poi_dict = {}
        poi_dict['name'] = poi['venue']['name']
        poi_dict['address'] = ', '.join(poi['venue']['location'].get('formattedAddress', []))
        poi_dict['fsq_id'] = poi['venue']['id']

        # Get the venue details from the Foursquare API
        venue_details = requests.get(f"https://api.foursquare.com/v2/venues/{poi_dict['fsq_id']}", params={
            'client_id': api_key.split(':')[0],
            'client_secret': api_key.split(':')[1],
            'v': '20230422'
        }).json()

        # Extract the venue ratings and reviews
        poi_dict['rating'] = venue_details['response']['venue'].get('rating', None)
        poi_dict['reviews'] = [review['text'] for review in venue_details['response']['venue'].get('tips', {}).get('groups', [])[0].get('items', [])]

        # Extract the venue contact information
        poi_dict['phone'] = venue_details['response']['venue'].get('contact', {}).get('phone', None)
        poi_dict['website'] = venue_details['response']['venue'].get('url', None)
        poi_dict['poi_type'] = [category['name'] for category in poi['venue']['categories']]
        poi_dict['location_name'] = location_name

        poi_list.append(poi_dict)
    return poi_list


In [4]:
# read the file containing the locations
venue_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

rand_df = venue_df.sample(n=3)

# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()

# Initialize an empty list to store the dictionaries
data_list = []

for _, row in tqdm(rand_df.iterrows(), total=rand_df.shape[0]):
    result = foursquare_query(row['latitude'], row['longitude'], row['name'])
    print(result)
    for venue in result['response']['venues']:
        # Retrieve additional information for the venue using the venue ID
        venue_id = venue['id']
        data_list.append(venue_details)

  0%|          | 0/3 [00:01<?, ?it/s]

{'results': [{'fsq_id': '4ac518bef964a520f0a220e3', 'categories': [{'id': 13018, 'name': 'Pub', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_', 'suffix': '.png'}}, {'id': 13142, 'name': 'English Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 32, 'geocodes': {'main': {'latitude': 51.496882, 'longitude': -0.192616}, 'roof': {'latitude': 51.496882, 'longitude': -0.192616}}, 'link': '/v3/places/4ac518bef964a520f0a220e3', 'location': {'address': '37 Marloes Rd', 'admin_region': 'England', 'country': 'GB', 'formatted_address': '37 Marloes Rd, London, Greater London, W8 6LA', 'locality': 'London', 'post_town': 'London', 'postcode': 'W8 6LA', 'region': 'Greater London'}, 'name': 'The Devonshire Arms', 'related_places': {}, 'timezone': 'Europe/London'}, {'fsq_id': '4cd6ae0adfb4a1cd4b48605c', 'categories': [{'id': 17068, 'name': 'Gourmet Store', 'icon': {'prefix': 'https://ss3.4sqi

KeyError: 'response'